In [1]:
%run 00_display_tables.ipynb 

Successfully connected to py_brew database.
+-------------+-------------------+---------------+----------------+
| customer_id |   customer_name   | customer_city | customer_state |
+-------------+-------------------+---------------+----------------+
|     100     | Jennifer Gonzalez |   Smithland   |    Arkansas    |
+-------------+-------------------+---------------+----------------+
+---------+---------------------+------------+-------------+-------+------+
| date_id |        tmstp        |    date    | day_of_week | month | year |
+---------+---------------------+------------+-------------+-------+------+
|    1    | 2020-07-04 21:23:10 | 2020-07-04 |      4      |   7   | 2020 |
+---------+---------------------+------------+-------------+-------+------+
+------------------+------------------+---------------------+--------------+---------------+
|    product_id    | product_category | product_subcategory | product_name | product_price |
+------------------+------------------+------

#### Calculate the average sales and total sales amount for each product category, including only the products that have been sold more than 10 times. Order the results by the total sales amount in descending order.

##### *Hint: Use a combination of aggregations, subqueries, and filtering conditions to calculate the average sales amount and include only the desired products.*

In [23]:
%%sql 

-- step 1: Create the subquery logic to select the distinct products
SELECT product_id
FROM fact_sales
GROUP BY product_id
HAVING COUNT(DISTINCT sales_id ) > 10
LIMIT 3 -- limit the output to 3 for visual

product_id
COL:ICE:COL:1051
COL:ICE:COL:1105
COL:ICE:COL:1139


In [13]:
%%sql 
-- step 2: Join the subquery 

SELECT p.product_category, 
AVG(s.sales_amount) AS average_sales, 
SUM(s.sales_amount) AS total_sales
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
JOIN (
    SELECT product_id
    FROM fact_sales
    GROUP BY product_id
    HAVING COUNT(DISTINCT sales_id) > 10
 ) AS subquery ON p.product_id = subquery.product_id
GROUP BY p.product_category
ORDER BY total_sales DESC 

product_category,average_sales,total_sales
Snacks,27.624575,39309.77
Retail,28.007972,38538.97
Cold Beverages,27.584482,37542.48
Hot Beverages,27.437614,35998.15


In [16]:
%%sql 

-- using CTE

WITH distinct_sales_CTE AS (
    SELECT product_id
    FROM fact_sales
    GROUP BY product_id
    HAVING COUNT(DISTINCT sales_id) > 10
) SELECT p.product_category,
AVG(s.sales_amount) AS average_sales,
SUM(s.sales_amount) AS total_sales
FROM fact_sales s 
INNER JOIN dim_product p ON p.product_id = s.product_id
JOIN distinct_sales_CTE dist ON dist.product_id = p.product_id
GROUP BY p.product_category
ORDER BY total_sales DESC;

product_category,average_sales,total_sales
Snacks,27.624575,39309.77
Retail,28.007972,38538.97
Cold Beverages,27.584482,37542.48
Hot Beverages,27.437614,35998.15


In [24]:
%%sql 

-- using inner query. This will result in the worst performance. Why?

SELECT p.product_category, 
AVG(s.sales_amount) AS average_sales, 
SUM(s.sales_amount) AS total_sales
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
WHERE p.product_id IN (
    SELECT product_id
    FROM fact_sales
    GROUP BY product_id
    HAVING COUNT(DISTINCT sales_id) > 10
 )
GROUP BY p.product_category
ORDER BY total_sales DESC 

product_category,average_sales,total_sales
Snacks,27.624575,39309.77
Retail,28.007972,38538.97
Cold Beverages,27.584482,37542.48
Hot Beverages,27.437614,35998.15


In [25]:
%%sql 

EXPLAIN SELECT p.product_category, 
AVG(s.sales_amount) AS average_sales, 
SUM(s.sales_amount) AS total_sales
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
WHERE p.product_id IN (
    SELECT product_id
    FROM fact_sales
    GROUP BY product_id
    HAVING COUNT(DISTINCT sales_id) > 10
 )
GROUP BY p.product_category
ORDER BY total_sales DESC 

id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
1,PRIMARY,p,None,ALL,PRIMARY,None,None,None,1000,100.0,Using where; Using temporary; Using filesort
1,PRIMARY,s,None,ref,product_id,product_id,82,py_brew.p.product_id,9,100.0,None
2,SUBQUERY,fact_sales,None,range,product_id,product_id,86,None,9741,100.0,Using index for group-by (scanning)


Let's analyze the values in each column in this **EXPLAIN SELECT**:

- `id`: The sequential number of the table within the query.The `PRIMARY` select_type indicates the main query.

- `select_type`: The type of select operation performed.

- `table`: The table involved in the operation.The `PRIMARY` select_type indicates the main query.

- `partitions`: The partitions used in the operation.Both tables have no partitions (`None`).

- `type`: The join type or access method used.The `type` column shows that a full table scan (`ALL`) is performed on the `p` table, which means it scans the entire table.

- `possible_keys`: The possible indexes that could be used.The `possible_keys` column shows that there are no possible indexes for the operation.

- `key`: The index used for the operation. The `key` column is `None`, indicating that no index is used.

- `key_len`: The length of the key used. The `key_len` column is also `None` as no key is used.

- `ref`: The column or expression used with the key. The `ref` column is `None` as there is no reference column used.

- `rows`: The estimated number of rows examined for the operation. The `rows` column shows an estimated number of 1000 rows examined for the operation.

- `filtered`: The percentage of rows filtered by the table condition. The `filtered` column indicates that 100% of the rows pass the table condition.
    
- `Extra`: Additional information or optimizations applied. The `Extra` column shows that temporary and filesort operations are used.


Based on this information, it appears that the query is performing a full table scan on the `p` table and using temporary and filesort operations. This indicates that there may be room for optimization, such as adding appropriate indexes on the join and filter columns to improve performance.